## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
#application_df['SPECIAL_CONSIDERATIONS'].value_counts()

In [5]:
# Removing the columns STATUS and SPECIAL_CONSIDERATIONS
#application_df = application_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1)
#application_df

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13', 'T12', 'T2', 'T25','T14','T15','T29','T17', 'T8','T7','T10']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2570        1
C1732        1
C4120        1
C1283        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C1234        2
C1256        2
C3200        2
C1246        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<2000].to_frame()
classifications.columns = ['count']
classifications = classifications.reset_index()
classifications_to_replace = classifications['index']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
X_dummies = pd.get_dummies(application_df)
print(X_dummies.columns)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_Other',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSI

In [12]:
# Split our preprocessed data into our features and target arrays
y = X_dummies['IS_SUCCESSFUL']
X = X_dummies.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(25724, 38)

## Compile, Train and Evaluate the Model

### Attempt 1
Changing the activation functions in the 3 hidden layers and 4 neurons per layer

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=4, activation="relu", input_dim=38))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=4, activation="tanh", input_dim=38))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=4, activation="exponential", input_dim=38))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 2s 1ms/step - loss: 0.6375 - accuracy: 0.6670
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5766 - accuracy: 0.7201
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5720 - accuracy: 0.7232
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5701 - accuracy: 0.7254
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5687 - accuracy: 0.7258
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5679 - accuracy: 0.7254
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5668 - accuracy: 0.7267
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5656 - accuracy: 0.7258
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5649 - accuracy: 0.7268
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5646 - accuracy: 0.7259

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 0s 914us/step - loss: 0.5651 - accuracy: 0.7238
Loss: 0.5651416778564453, Accuracy: 0.7238484025001526


### Attempt 2
2 Hidden layers with both having relu activation functions and 12 neurons

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=38))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=38))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 12)                468       
_________________________________________________________________
dense_5 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 13        
Total params: 637
Trainable params: 637
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.6003 - accuracy: 0.7026
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5651 - accuracy: 0.7243
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5619 - accuracy: 0.7256
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5598 - accuracy: 0.7272
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5586 - accuracy: 0.7273
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5584 - accuracy: 0.7282
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5574 - accuracy: 0.7280
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7291
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5561 - accuracy: 0.7293
Epoch 10/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5561 - accuracy: 0.7289

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 0s 895us/step - loss: 0.5625 - accuracy: 0.7254
Loss: 0.5624662041664124, Accuracy: 0.7253644466400146


### Attempt 3
4 Hidden layers with more number of neurons in each layer

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="tanh", input_dim=38))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="tanh", input_dim=38))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim=38))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 40)                1560      
_________________________________________________________________
dense_8 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_9 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 168       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 3,587
Trainable params: 3,587
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=120)

Epoch 1/120
804/804 [==============================] - 2s 1ms/step - loss: 0.5796 - accuracy: 0.7152
Epoch 2/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5652 - accuracy: 0.7260
Epoch 3/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5623 - accuracy: 0.7281
Epoch 4/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5605 - accuracy: 0.7295
Epoch 5/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5590 - accuracy: 0.7295
Epoch 6/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5574 - accuracy: 0.7297
Epoch 7/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7313
Epoch 8/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5554 - accuracy: 0.7308
Epoch 9/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7318
Epoch 10/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5418 - accuracy: 0.7385
Epoch 82/120
804/804 [==============================] - 1s 2ms/step - loss: 0.5418 - accuracy: 0.7381
Epoch 83/120
804/804 [==============================] - 1s 2ms/step - loss: 0.5421 - accuracy: 0.7368
Epoch 84/120
804/804 [==============================] - 1s 2ms/step - loss: 0.5419 - accuracy: 0.7380
Epoch 85/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5418 - accuracy: 0.7380
Epoch 86/120
804/804 [==============================] - 1s 2ms/step - loss: 0.5415 - accuracy: 0.7378
Epoch 87/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7376
Epoch 88/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7374
Epoch 89/120
804/804 [==============================] - 1s 2ms/step - loss: 0.5416 - accuracy: 0.7375
Epoch 90/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5415 - accuracy: 

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 0s 993us/step - loss: 0.5690 - accuracy: 0.7250
Loss: 0.5689603686332703, Accuracy: 0.7250145673751831
